In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
from bs4 import BeautifulSoup as BS
import re
import bs4
import csv
from extraction import extract_data_1, extract_data_2
import json

In [3]:
with open("headings.csv") as fd:
    reader = csv.reader(fd)
    structure = {}
    for line in reader:
        try:
            structure[line[0]] = int(line[1])
        except:
            pass



In [4]:
# Get the list of files in rendered dir
ren_dir = "rendered"
files = glob.glob(os.path.join(ren_dir, "*.html"))
class_name = re.compile(r'class="([\w]+_light)"')


In [5]:
data = {}

In [6]:
for k, v in structure.items():
    data[k] = {}
    data[k]["data"] = {}

In [7]:
for f in files:
    with open(f, encoding="mbcs") as fd:
        html_str = fd.read()
        soup = BS(html_str)
        country_name = soup.h2.span.get_text()
        for k, v in structure.items():
            data[k]["data"][country_name] = {}
        class_result = class_name.search(html_str)
        heading_nodes = soup.find_all('tr', class_=class_result.group(1))
        headings = list(
            map(lambda x: re.sub(r"-+", "-", (x.a.get_text()[1:]).replace(":", "").lower().replace(" ", "-").replace("(", "").replace(")", "")), heading_nodes))
        values = list(map(lambda x: x.find_next_sibling(), heading_nodes))
        
        country_data = []
        
        for val in values:
            try:
                cat_data = []
                for child in val.td.children:
                    if not type(child) == bs4.element.NavigableString:
                        cat_data.append(re.sub(r"[\s]+", " ", child.get_text()).strip())
                country_data.append(cat_data)
            except Exception as ex:
                print(ex)
        
        for i in range(len(headings)):
            head = headings[i]
            try:
                if structure[head] == 1:
                    vals, unit = extract_data_1(country_data[i])
                    if not data[head].get("unit") and not unit == "":
                        data[head]["unit"] = unit
                    for val in vals:
                        data[head]["data"][country_name][val[1]] = val[0]
                elif structure[head] == 2:
                    vals, unit, year = extract_data_2(country_data[i])
                    data[head]["data"][country_name][year] = {}
                    if not data[head].get("unit") and not unit == "":
                        data[head]["unit"] = unit
                    for val in vals:
                        data[head]["data"][country_name][year][val[0]] = val[1]
            except KeyError:
                continue
            except Exception as ex:
                print(ex)

Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not Found
Year Not F

In [8]:
data

{'land-use': {'data': {'Aruba': {2005: {'arable-land': 11.11,
     'permanent-crops': 0.0,
     'other': 88.89}},
   'Antigua and Barbuda': {2011: {'arable-land': 9.09,
     'permanent-crops': 2.27,
     'other': 88.64}},
   'United Arab Emirates': {2011: {'arable-land': 0.61,
     'permanent-crops': 0.5,
     'other': 98.9}},
   'Afghanistan': {2011: {'arable-land': 11.95,
     'permanent-crops': 0.18,
     'other': 87.87}},
   'Algeria': {2011: {'arable-land': 3.15,
     'permanent-crops': 0.38,
     'other': 96.46}},
   'Azerbaijan': {2011: {'arable-land': 21.78,
     'permanent-crops': 2.62,
     'other': 75.6}},
   'Albania': {2011: {'arable-land': 21.63,
     'permanent-crops': 2.57,
     'other': 75.79}},
   'Armenia': {2011: {'arable-land': 14.47,
     'permanent-crops': 1.8,
     'other': 83.74}},
   'Andorra': {2011: {'arable-land': 5.32,
     'permanent-crops': 0.0,
     'other': 94.68}},
   'Angola': {2011: {'arable-land': 3.29,
     'permanent-crops': 0.23,
     'other': 9

In [9]:
with open("dump2014.json", "w") as fd:
    json.dump(data, fd)